In [ ]:
using  SymPy
using  Plots; pyplot()

In [ ]:
const η = 1e-10
const h = 0.005
const ht = 0.025
const wd = 5*h
const lsp = 0.010
const rNum = 0.25 - 0.5*wd
const RNum = 0.50

In [ ]:
x,x₁,y,y₁,r,r₁,k,t = symbols("x,x₁,y,y₁,r,r₁,k,t", real = true)

In [ ]:
Eᵣ = 40e3
Eₘ = 4e3
Eᵢ = 4e3
νᵣ = 0.33
νₘ = 0.4
νᵢ = 0.4

@show λᵣ = Eᵣ*νᵣ/((1+νᵣ)*(1-2*νᵣ))
@show λₘ = Eₘ*νₘ/((1+νₘ)*(1-2*νₘ))
@show λᵢ = Eᵢ*νᵢ/((1+νᵢ)*(1-2*νᵢ))

@show μᵣ = Eᵣ/(2*(1+νᵣ))
@show μₘ = Eₘ/(2*(1+νₘ))
@show μᵢ = Eᵢ/(2*(1+νᵢ))

In [ ]:
Gᵢᵢ = 0.05
Gₘ = 0.25
Gᵣ = 0.50
Gᵢ = Gᵢᵢ

In [ ]:
f = (wd/lsp)^3 - 3*(wd/lsp)^2 + 3*(wd/lsp)

### Piecewise function

In [ ]:
heaviside(x) = 0.5 * (sign(x) + 1)

interval(x, a, b) = heaviside(x-a) - heaviside(x-b)

In [ ]:
s = abs(r₁-r)/h;
w(r,r₁) = (1/(sqrt(2*π)*h))*exp(-s^2/2);

In [ ]:
λₕ(r) = λᵣ*interval(r,-rNum,rNum) - λₘ*interval(r,-rNum-wd,rNum+wd) + λᵢ* interval(r,-rNum-wd,rNum+wd) - λᵢ* interval(r,-rNum,rNum) + λₘ
smoothλ = integrate(λₕ(r₁)*w(r,r₁),(r₁,-2*RNum-ht,2*RNum+ht))
λ = simplify(smoothλ.subs(r, sqrt(x^2+y^2)))

In [ ]:
μₕ(r) = μᵣ*interval(r,-rNum,rNum) - μₘ*interval(r,-rNum-wd,rNum+wd) + μᵢ* interval(r,-rNum-wd,rNum+wd) - μᵢ* interval(r,-rNum,rNum) + μₘ
smoothμ = integrate(μₕ(r₁)*w(r,r₁),(r₁,-2*RNum-ht,2*RNum+ht))
μ = simplify(smoothμ.subs(r, sqrt(x^2+y^2)))

In [ ]:
Gₕ(r) = Gᵣ*interval(r,-rNum,rNum) - Gₘ*interval(r,-rNum-wd,rNum+wd) + Gᵢ* interval(r,-rNum-wd,rNum+wd) - Gᵢ* interval(r,-rNum,rNum) + Gₘ
smoothG = integrate(Gₕ(r₁)*w(r,r₁),(r₁,-2*RNum-ht,2*RNum+ht))
G = (smoothG.subs(r, sqrt(x^2+y^2)))

In [ ]:
discontλₕ(x,y) = (λₕ(r)).subs(r, sqrt(x^2+y^2))
discontμₕ(x,y) = (μₕ(r)).subs(r, sqrt(x^2+y^2))
discontGₕ(x,y) = (Gₕ(r)).subs(r, sqrt(x^2+y^2))

In [ ]:
xs = ys  = range(-RNum,RNum, length=100)
plot1 = contourf(xs,ys,discontλₕ, c=:diverging_bwr_40_95_c42_n256, axis = nothing, showaxis =false,aspect_ratio=:equal, camera=(0,90))
plot2 = contourf(xs,ys,λ, c=:diverging_bwr_40_95_c42_n256, axis = nothing, showaxis =false,aspect_ratio=:equal, camera=(0,90))
plot(plot1,plot2)

In [ ]:
using Gmsh:gmsh
using Gridap
using GridapGmsh
using Gridap.TensorValues

In [ ]:
using Gridap.Arrays
using Gridap.Fields
using Gridap.ReferenceFEs
using Gridap.Geometry
using Gridap.CellData
using Gridap.Algebra
using LinearAlgebra

## Creating mesh for the geometry

In [ ]:
hf = lsp/2.1
hd = 30*hf
Lₚ = 1.0
Hₚ = 1.0
rc = 0.25
rc1 = 0.25 - 0.5*wd
rc2 = 0.25 + 0.5*wd

In [ ]:
gmsh.initialize()
gmsh.option.setNumber("General.Terminal", 1)


gmsh.model.geo.addPoint(-Lₚ/2, -Hₚ/2, 0.0, hd, 1)
gmsh.model.geo.addPoint(Lₚ/2, -Hₚ/2, 0.0, hd, 2)
gmsh.model.geo.addPoint(Lₚ/2, Hₚ/2, 0.0, hd, 3)
gmsh.model.geo.addPoint(-Lₚ/2, Hₚ/2, 0.0, hd, 4)

gmsh.model.geo.addPoint(0, Hₚ/2, 0.0, hf, 100)
gmsh.model.geo.addPoint(0, -Hₚ/2, 0.0, hf, 101)

gmsh.model.geo.addLine(1, 101, 1)
gmsh.model.geo.addLine(101, 2, 101)
gmsh.model.geo.addLine(2, 3, 2)
gmsh.model.geo.addLine(3, 100, 102)
gmsh.model.geo.addLine(100, 4, 3)
gmsh.model.geo.addLine(4, 1, 4)
gmsh.model.geo.addCurveLoop([3,4,1,101,2,102],1)
gmsh.model.addPhysicalGroup(2, [1],1)

gmsh.model.geo.addPoint(-rc1, 0.0, 0.0, hf, 5)
gmsh.model.geo.addPoint(rc1, 0.0, 0.0, hf, 6)
gmsh.model.geo.addPoint(0.0, 0.0, 0.0, hf, 7)
gmsh.model.geo.addCircleArc(5, 7, 6, 5)
gmsh.model.geo.addCircleArc(6, 7, 5, 6)
gmsh.model.geo.addCurveLoop([5,6],2)
gmsh.model.addPhysicalGroup(2, [2],2)
gmsh.model.geo.addPlaneSurface([2], 2)

gmsh.model.geo.addPoint(-rc2, 0.0, 0.0, hf, 8)
gmsh.model.geo.addPoint(rc2, 0.0, 0.0, hf, 9)
gmsh.model.geo.addPoint(0.0, 0.0, 0.0, hf, 10)
gmsh.model.geo.addCircleArc(8, 10, 9, 7)
gmsh.model.geo.addCircleArc(9, 10, 8, 8)
gmsh.model.geo.addCurveLoop([7,8],3)
gmsh.model.geo.addPlaneSurface([2,-3], 3)
gmsh.model.addPhysicalGroup(2, [3],3)

gmsh.model.geo.addPlaneSurface([3,-1], 1)

gmsh.model.addPhysicalGroup(0, [1],1)
gmsh.model.addPhysicalGroup(1, [4],1)
gmsh.model.addPhysicalGroup(1, [2],2)

gmsh.model.setPhysicalName(2, 1, "Matrix")
gmsh.model.setPhysicalName(2, 2, "InHom")
gmsh.model.setPhysicalName(2, 3, "Interface")

gmsh.model.setPhysicalName(0, 1, "DirichletBot")
gmsh.model.setPhysicalName(1, 1, "DirichletLeft")
gmsh.model.setPhysicalName(1, 2, "DirichletRight")


gmsh.model.mesh.field.add("Box", 11)
gmsh.model.mesh.field.setNumber(11, "VIn", hf)
gmsh.model.mesh.field.setNumber(11, "VOut", hd)
gmsh.model.mesh.field.setNumber(11, "XMin", -0.05*Lₚ)
gmsh.model.mesh.field.setNumber(11, "XMax", 0.35*Lₚ)
gmsh.model.mesh.field.setNumber(11, "YMin", -Hₚ/2)
gmsh.model.mesh.field.setNumber(11, "YMax", Hₚ/2)
gmsh.model.mesh.field.setAsBackgroundMesh(11)

gmsh.model.geo.synchronize()
gmsh.model.mesh.generate(2)

gmsh.write("MatrixWithSingleFiber.msh")
gmsh.finalize()

In [ ]:
model = GmshDiscreteModel("MatrixWithSingleFiber.msh")
writevtk(model,"MatrixWithSingleFiber")

In [ ]:
order = 1
degree = 2*order
Ω = Triangulation(model)
dΩ = Measure(Ω,degree)

In [ ]:
using Gridap.Geometry
labels = get_face_labeling(model)
dimension = 2
mat_tags = get_face_tag(labels,dimension);

In [ ]:
p = get_cell_points(Ω)

In [ ]:
reffe = ReferenceFE(lagrangian,Float64,order)
V₁ = FESpace(model, reffe, conformity=:H1)

In [ ]:
V₂ = FESpace(model, reffe, conformity=:H1)
V₃ = FESpace(model, reffe, conformity=:H1)

In [ ]:
V₁₂ = MultiFieldFESpace([V₁,V₁,V₁])

In [ ]:
lambdify(λ)

In [ ]:
λn = lambdify(λ)
μn = lambdify(μ)
Gn = lambdify(G)

In [ ]:
λh2(p) = λn.(p[1],p[2])
μh2(p) = μn.(p[1],p[2])
Gh2(p) = Gn.(p[1],p[2])

In [ ]:
λ_h = interpolate_everywhere(λh2,V₁)
μ_h = interpolate_everywhere(μh2,V₂)
G_h = interpolate_everywhere(Gh2,V₃)

In [ ]:
writevtk(Ω,"MaterialProperties",cellfields= ["λ"=>λ_h,"μ"=>μ_h,"Gc"=> G_h])

In [ ]:
function σ_mod(ε, ε_in, s_in,λ,μ)
    σ_elas = λ*tr(ε)*one(ε) + 2*μ*ε
    σ = (s_in^2 + η)*σ_elas
    return σ
end

In [ ]:
function ψPos(ε_in,λ,μ)
    σ_elas = λ*tr(ε_in)*one(ε_in) + 2*μ*ε_in
    σ_array = get_array(σ_elas)
    Egn_Vals = eigvals(σ_array)
    Egn_Vecs = eigvecs(σ_array)
    σ1 = Egn_Vals[1]
    σ2 = Egn_Vals[2]
    n1 = VectorValue(Egn_Vecs[:,1])
    n2 = VectorValue(Egn_Vecs[:,2])
 if (σ1)  >= 0
        if (σ2)  >= 0
            ψPlus = 0.5*(ε_in ⊙ (σ1*(n1 ⊗ n1) + σ2*(n2 ⊗ n2)))
        else
            ψPlus = 0.5*(ε_in ⊙ (σ1*(n1 ⊗ n1)))
        end
    elseif (σ1)  < 0
        if (σ2)  >= 0
            ψPlus = 0.5*(ε_in ⊙ (σ2*(n2 ⊗ n2)))
        else
            ψPlus = 0.0
        end
    end
    return ψPlus
end

In [ ]:
function new_EnergyState(ψPlusPrev_in,ψhPos_in)
  ψPlus_in = ψhPos_in
  if ψPlus_in >= ψPlusPrev_in
    ψPlus_out = ψPlus_in
  else
    ψPlus_out = ψPlusPrev_in
  end
  true,ψPlus_out
end

In [ ]:
function project(q,model,dΩ,order)
  reffe = ReferenceFE(lagrangian,Float64,order)
  V = FESpace(model,reffe,conformity=:L2)
  a(u,v) = ∫( u*v )*dΩ
  l(v) = ∫( v*q )*dΩ
  op = AffineFEOperator(a,l,V,V)
  qh = Gridap.solve(op)
  qh
end

In [ ]:
sId = CellState(1.0,dΩ)
shId = project(sId,model,dΩ,order)

In [ ]:
LoadTagId = get_tag_from_name(labels,"DirichletRight")
Γ_Load = BoundaryTriangulation(model,tags = LoadTagId)
dΓ_Load = Measure(Γ_Load,degree)
n_Γ_Load = get_normal_vector(Γ_Load)

In [ ]:
reffe_PF = ReferenceFE(lagrangian,Float64,order)
V0_PF = TestFESpace(model,reffe_PF;
  conformity=:H1)
U_PF = TrialFESpace(V0_PF)
sh = zero(V0_PF)

In [ ]:
reffe_Disp = ReferenceFE(lagrangian,VectorValue{2,Float64},order)
        V0_Disp = TestFESpace(model,reffe_Disp;
          conformity=:H1,
          dirichlet_tags=["DirichletBot","DirichletLeft","DirichletRight"],
          dirichlet_masks=[(true,true), (true,false), (true,false)])

uh = zero(V0_Disp)

In [ ]:
function   stepPhaseField(uh_in ,ΨPlusPrev_in)
    a_PF(s,φ) =∫( G_h*lsp*∇(φ)⋅∇(s)+ 2*ΨPlusPrev_in*s*φ+ (G_h/lsp)*s*φ)*dΩ
    b_PF(φ) =∫( (G_h/lsp)*φ)*dΩ
    op_PF = AffineFEOperator(a_PF,b_PF ,U_PF ,V0_PF)
    sh_out = Gridap.solve(op_PF)
    return  sh_out
end

In [ ]:
function   stepDisp(uh_in,sh_in,uApp)
        uApp1(x) = VectorValue(0.0,0.0)
        uApp2(x) = VectorValue(0.0,0.0)
        uApp3(x) = VectorValue(uApp,0.0)
        U_Disp = TrialFESpace(V0_Disp,[uApp1,uApp2,uApp3])

        a_Disp(u,v) = ∫( (ε(v) ⊙ (σ_mod∘(ε(u),ε(uh_in),sh_in,λ_h,μ_h)) ) )*dΩ
        b_Disp(v) = 0.0
        op_Disp = AffineFEOperator(a_Disp,b_Disp,U_Disp,V0_Disp)
        uh_out = Gridap.solve(op_Disp)
    return uh_out
end

In [ ]:
uApp = 0
delu = 2e-4
uAppMax = 0.020
innerMax = 10
count = 0
Load = Float64[]
Displacement = Float64[]
push!(Load, 0.0)
push!(Displacement, 0.0)

sPrev = CellState(1.0,dΩ)
sh = project(sPrev,model,dΩ,order)
ψPlusPrev = CellState(0.0,dΩ)

while uApp .< uAppMax 
        
    uApp = uApp .+ delu
    count = count .+ 1
    
    if uApp >= 0.005
        delu = 1.5e-4
    end
   
    print("\n Entering displacemtent step$count :", float(uApp))
    
   for inner = 1:innerMax
        
        ψhPlusPrev = project(ψPlusPrev,model,dΩ,order)
        
        RelErr = abs(sum(∫( G_h*lsp*∇(sh)⋅∇(sh) + 2*ψhPlusPrev*sh*sh + (G_h/lsp)*sh*sh)*dΩ-∫( (G_h/lsp)*sh)*dΩ))/abs(sum(∫( (G_h/lsp)*sh)*dΩ))

        print("\n error = ",float(RelErr))
        
        sh = stepPhaseField(uh,ψhPlusPrev) 
        uh = stepDisp(uh,sh,uApp)
        
        ψhPos_in = ψPos∘(ε(uh),λ_h,μ_h) 
        update_state!(new_EnergyState,ψPlusPrev,ψhPos_in)
        
        if RelErr < 1e-8
            break 
        end 
    end
    
    Node_Force = sum(∫( n_Γ_Load ⋅ (σ_mod∘(ε(uh),ε(uh),sh,λ_h,μ_h)) ) *dΓ_Load)
    
    push!(Load, Node_Force[1])
    push!(Displacement, uApp)
    if uApp <= 0.01
        if mod(count,2) == 0 
             writevtk(Ω,"results_PhaseFieldMatrixWithCircularFiber$count",cellfields=
            ["uh"=>uh,"s"=>sh , "epsi"=>ε(uh)])
        end
    elseif uApp > 0.01
             writevtk(Ω,"results_PhaseFieldMatrixWithCircularFiber$count",cellfields=
            ["uh"=>uh,"s"=>sh , "epsi"=>ε(uh)])
    end
end 

In [ ]:
writevtk(Ω,"results_PhaseFieldMatrixWithCircularFiber$count",cellfields=
        ["uh"=>uh,"s"=>sh , "epsi"=>ε(uh)])

In [ ]:
plot(Displacement,Load,label = nothing)
xlabel!("Displacement (mm)")
ylabel!("Load (N)")

In [ ]:
using DelimitedFiles
Disp = writedlm("DispDataMatrixWithCircularInclusion.csv",  Displacement, ',')
Force = writedlm("LoadDataMatrixWithCircularInclusion.csv",  Load, ',')